## Meta information
* train instances: "/home/shiina/data/nips/vqa2/annotation/cache/train_instances_with_doublehints.pkl"
* val instances: "/home/shiina/data/nips/vqa2/annotation/cache/val_instances_with_doublehints.pkl"

In [1]:
import torch, copy
import torchtext.vocab as vocab
glove_text = vocab.GloVe(name='6B', dim=300)
print(glove_text["animal"].shape)

torch.Size([300])


In [2]:
from stanfordcorenlp import StanfordCoreNLP
nlp_parser = StanfordCoreNLP('http://localhost', port=9000, timeout=300000)
print(nlp_parser)


In [3]:
print(nlp_parser)
vg_replace_dict = {
    "ceiling fan": "fan",
    "birthday cake": "cake",
    "skateboard ramp": "ramp",
    "towel rack": "rack",
    "tree branch": "branch",
    "tile floor": "floor",
    "ski jacket": "anorak",
    "tennis court": "court",
    "rock wall": "wall",
    "tennis racket,tennis racquet": "racquet",
    "toilet brush": "brush",
    "wii remote": "remote-control",
    "brocolli": "broccoli",
    "sandwhich": "sandwich",
    "skiis": "skis",
    "kneepad": "kneecap"
}
print("kneecap" in glove_text.stoi)
print("skis" in glove_text.stoi)

True
True


In [4]:

similarity_func = glove_text

def load_class():
    ret = {}
    with open("/home/shiina/shiina/question/aaai/preprocess/vqa2/vg_class.txt", "r") as f:
        lines = f.readlines()
        for idx, cls in enumerate(lines):
            cls = cls.strip()
#             print(idx, cls)
            candidate = cls.split(",")
            is_find = False
            for can in candidate:
                
                can_ = can.replace(" ", "-")
                can_noblank = can.replace(" ", "")
                if can in similarity_func.stoi:
                    ret[idx] = [can]
                    is_find = True
                    # print("11111")
                    break
                if can_ in similarity_func.stoi:
                    ret[idx] = [can_]
                    is_find = True
                    # print("11111")
                    break
                if can_noblank in similarity_func.stoi:
                    ret[idx] = [can_noblank]
                    is_find = True
                    # print("11111")
                    break
                elif can in vg_replace_dict.keys():
                    ret[idx] = vg_replace_dict[can]
                    is_find = True
                    print("2222")
                    break
            if not is_find:
                collect = []
                for can in candidate:
                    collect.extend(can.split(" "))
                valid_collect = []
                for can in collect:
                    if can in similarity_func.stoi:
                        valid_collect.append(can)
                if len(valid_collect) < 1:
                    print(idx, cls, "*******")
                ret[idx] = valid_collect
    return ret

#     return ret
vg_idx2cls = load_class()

2222
2222
2222
2222
2222
2222
2222
2222
2222
2222
2222
2222
2222
2222


In [5]:
def l2_similarity(a, b):
    if not isinstance(a, torch.Tensor):
        a = torch.from_numpy(a).view(-1)
        b = torch.from_numpy(b).view(-1)
    diff = a - b
    l2 = torch.norm(diff, dim=0)
    return l2

def l2_similarity_2d(a, b):
    assert len(a.shape) == 2
    assert len(b.shape) == 2
    assert a.shape[1] == b.shape[1]
    a = a.unsqueeze(1) # N1, 1, D
    b = b.unsqueeze(0) # 1, N2, D
    diff = a - b
    l2 = torch.norm(diff, dim=2)
    return l2

similarity_func = glove_text

def similarity(candidate_word, reference_word_list, threshold=5.7):
    ret = []
    for ref in reference_word_list:
        ref_vectors = [similarity_func[ref_word].unsqueeze(0) for ref_word in ref]
        ref_vectors = torch.cat(ref_vectors, dim=0)
        candidate_vector = vector = similarity_func[candidate_word].unsqueeze(0)
        similarity_matrix = l2_similarity_2d(candidate_vector, ref_vectors)
        most_similar_value = similarity_matrix[0, :].min()
        if most_similar_value.item() <= threshold:
            ret.append(1)
        else:
            ret.append(0)
    if len(ret) < 36:
        ret.extend([0 for _ in range(36 - len(ret))])
    return torch.Tensor(ret)

In [28]:
import re, torch, collections, json

def stanfordcorenlp_pos_tag(sentence: str, nlp_processor):
    sentence = re.sub('\.+', r'.', sentence)
    sentence = re.sub('([a-z])([.,!?()])', r'\1 \2 ', sentence)
    sentence = re.sub('\?', ' ', sentence)
    sentence = re.sub('\s+', ' ', sentence)
    props = {
        'annotators': 'ssplit,tokenize,pos,lemma',
        "tokenize.options":
            "splitHyphenated=false,normalizeParentheses=false,normalizeOtherBrackets=false",
        "tokenize.whitespace": False,
        'ssplit.isOneSentence': True,
        'outputFormat': 'json'
    }
    ret = nlp_processor.annotate(sentence.lower(), props)
    try:
        pos_dict = json.loads(ret)['sentences'][0]['tokens']

    except:
        return []

    ret = []
    for token in pos_dict:
        ret.append({
            "word_idx": token['index'] - 1,
            "word": token["word"],
            "lemma": token["lemma"],
            "pos": token["pos"]
        })
    return ret


def align_sentence_bbox(parsed_sentence, bbox, threshold=5.7):

    bbox_cls = bbox["object_id"].reshape(-1).tolist()
#     print(bbox_cls)

    # convert class to vectors
#     print(vg_idx2cls[0])

    bbox_cls_name = [vg_idx2cls[bbox_cls_id] for bbox_cls_id in bbox_cls]
    
#     print(bbox_cls_name)

#     bbox_vectors = [class_vectors[bbox_cls_id].unsqueeze(0) for bbox_cls_id in bbox_cls]
#     bbox_vectors = torch.cat(bbox_vectors, dim=0)

    parsed_sentence = copy.deepcopy(parsed_sentence)

    mask = torch.zeros(36)
    for word_inst in parsed_sentence:
        word = word_inst["word"]
        lemma = word_inst["lemma"]
        pos = word_inst["pos"]
        if pos not in ["PRP", "PRP$"] and pos[:1] != "N":
            continue

        if pos in ["PRP", "PRP$"]:
            if lemma in ["he", "she"]:
                word = "person"
                lemma = "person"
        if lemma in ["he", "she", "boy", "girl", "his", "her", "woman", "man", "people", "crowd", "lady", "gentleman"]:
            lemma = "person"

        if lemma in similarity_func.stoi:
            valid_mask = similarity(lemma, bbox_cls_name, threshold)
            mask += valid_mask
    return mask > 0
from tqdm import tqdm

import cv2

def visualize(image_path, ppl_path):
    src_img = cv2.imread(image_path)
    
    with open(ppl_path, "rb") as f:
        ppl = pickle.load(f)
    
    for i in range(36):
        if ppl["confidence"][i][0] <= 0.4:
            break
        coor = ppl["original_spatial_features"][i, :]
        x1, y1, x2, y2 = coor
        cv2.rectangle(src_img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), thickness=2)
    
    bbox_cls = ppl["obj_cls"].reshape(-1).tolist()

    bbox_cls_name = [vg_idx2cls[bbox_cls_id] for bbox_cls_id in bbox_cls]
    print(bbox_cls_name)
    
    print(ppl["confidence"])

    output_name = "/home/shiina/output/test.jpg"
    cv2.imwrite(output_name, src_img)
    pass

MAX_VH = 4

def annotate(split: collections.defaultdict, threshold=5.7):
    ret = {}
    cnt = 0
    for qid, inst in tqdm(split.items()):
        question = inst["question"].lower()
        answer = inst["multiple_choice_answer"].lower()
        img_id = inst["image_id"]

        parsed_results = stanfordcorenlp_pos_tag(question, nlp_parser)
        if not parsed_results:
            continue
        cnt += 1
        ppl_path = inst["ppl_path"]
        with open(ppl_path, "rb") as f:
            bbox = pickle.load(f)
        
        question_valid_mask = align_sentence_bbox(parsed_results, bbox, threshold=threshold)
        question_tokens = [token_inst["word"] for token_inst in parsed_results]
        question_lemmas = [token_inst["lemma"] for token_inst in parsed_results]
        
        
        parsed_results = stanfordcorenlp_pos_tag(inst["multiple_choice_answer"].lower(), nlp_parser)
        if not parsed_results:
            continue
        answer_valid_mask = align_sentence_bbox(parsed_results, bbox, threshold=threshold)
        answer_tokens = [token_inst["word"] for token_inst in parsed_results]
        answer_lemmas = [token_inst["lemma"] for token_inst in parsed_results]

        valid_mask = (question_valid_mask.float() + answer_valid_mask.float()) > 0
        valid_mask = valid_mask.float()
        
        if valid_mask.sum() > MAX_VH:
#             print(img_id)
#             print(question)
#             print(answer)
#             print(valid_mask)
#             image_id_str = str(img_id).zfill(12)
#             visualize(os.path.join("/home/shiina/data/coco2014/train2014/pic", "COCO_train2014_{}.jpg".format(image_id_str)), ppl_path)
#             break
            # we will select at most 4 proposals, considering both diversity and confidence
#             print("before prune: ", valid_mask)
            active_list = torch.where(valid_mask == 1)
            active_list = active_list[0]
#             print(active_list, "ppppppp")
            
            confidence = bbox["confidence"]
            
            bbox_cls = bbox["object_id"].reshape(-1).tolist()
            
            candidate_score = [confidence[idx, 0] for idx in active_list]
            candidate_cls = copy.deepcopy([bbox_cls[idx] for idx in active_list])
#             print("candidate_score: ", candidate_score)
#             print("candidate_cls: ", candidate_cls)
            
            
            tobeselected_cls = list(set(candidate_cls))[:MAX_VH]
#             print("tobeselected_cls: ", tobeselected_cls)
            
            chosen = []
            
            for cls in tobeselected_cls:
                for i in range(len(active_list)):
                    if candidate_cls[i] == cls:
                        chosen.append(active_list[i])
                        candidate_cls[i] = -1
                        break
            if len(chosen) < MAX_VH:
                for i in range(len(active_list)):
                    if candidate_cls[i] == -1:
                        continue
                    chosen.append(active_list[i])
                    if len(chosen) >= MAX_VH:
                        break
            valid_mask = torch.zeros(36)
            for i in chosen:
                valid_mask[i] = 1
        
        if valid_mask.sum() == 0 and answer_tokens != ["no"]:
            continue
        else:
            
            inst["question_tokens"] = question_tokens
            inst["question_lemmas"] = question_lemmas
            inst["answer_tokens"] = answer_tokens
            inst["answer_lemmas"] = answer_lemmas
            inst["visual_hint"] = valid_mask.numpy().tolist()
            ret[qid] = copy.deepcopy(inst)
    
    print("Statistics: before processing: {}, after processing: {}".format(cnt, len(ret.keys())))
    return ret

In [29]:
import pickle, os

train_input_instances_path = "/home/shiina/data/nips/vqa2/annotation/raw/train_instances.pkl"
train_annotation_save_path = "/home/shiina/data/nips/vqa2/annotation/cache/train_instances_with_doublehints.pkl"

os.makedirs(os.path.split(train_annotation_save_path)[0], exist_ok=True)

with open(train_input_instances_path, "rb") as f:
    train_instances = pickle.load(f)
annotated_data = annotate(split=train_instances)

with open(train_annotation_save_path, "wb") as f:
    pickle.dump(annotated_data, f)
print("finish, save to {}".format(train_annotation_save_path))


100%|██████████| 443757/443757 [1:25:54<00:00, 86.10it/s]
Statistics: before processing: 443757, after processing: 355157
finish, save to /home/shiina/data/nips/vqa2/annotation/cache/train_instances_with_doublehints.pkl


In [30]:
val_input_instances_path = "/home/shiina/data/nips/vqa2/annotation/raw/val_instances.pkl"
val_annotation_save_path = "/home/shiina/data/nips/vqa2/annotation/cache/val_instances_with_doublehints.pkl"

os.makedirs(os.path.split(val_annotation_save_path)[0], exist_ok=True)

with open(val_input_instances_path, "rb") as f:
    val_instances = pickle.load(f)
val_annotated_data = annotate(split=val_instances)

with open(val_annotation_save_path, "wb") as f:
    pickle.dump(val_annotated_data, f)
print("finish, save to {}".format(val_annotation_save_path))

100%|██████████| 214354/214354 [40:48<00:00, 87.56it/s]
Statistics: before processing: 214354, after processing: 171978
finish, save to /home/shiina/data/nips/vqa2/annotation/cache/val_instances_with_doublehints.pkl
